# Self-Attention for Vision

For this assignment, we're going to implement self-attention blocks in a convolutional neural network for CIFAR-10 Classification.

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [ ]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./data/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./data/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./data/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/datasets/cifar-10-python.tar.gz to ./data/datasets
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


## Flatten Function

In [ ]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Check Accuracy Function


In [ ]:
import torch.nn.functional as F  # useful stateless functions
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

### Training Loop

In [ ]:
def train(model, optimizer, epochs=10):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    acc_max = 0
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                acc = check_accuracy(loader_val, model)
                if acc >= acc_max:
                    acc_max = acc
                print()
    print("Maximum accuracy attained: ", acc_max)

In [ ]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

## Vanilla CNN; No Attention
We implement the vanilla architecture for you here. Do not modify the architecture. You will use the same architecture in the following parts. Do not modify the hyper-parameters.

In [ ]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.2933
Checking accuracy on validation set
Got 110 / 1000 correct (11.00)

Epoch 0, Iteration 100, loss = 1.4623
Checking accuracy on validation set
Got 468 / 1000 correct (46.80)

Epoch 0, Iteration 200, loss = 1.4420
Checking accuracy on validation set
Got 500 / 1000 correct (50.00)

Epoch 0, Iteration 300, loss = 1.2049
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)

Epoch 0, Iteration 400, loss = 1.1457
Checking accuracy on validation set
Got 579 / 1000 correct (57.90)

Epoch 0, Iteration 500, loss = 1.3131
Checking accuracy on validation set
Got 580 / 1000 correct (58.00)

Epoch 0, Iteration 600, loss = 1.1777
Checking accuracy on validation set
Got 598 / 1000 correct (59.80)

Epoch 0, Iteration 700, loss = 1.1197
Checking accuracy on validation set
Got 603 / 1000 correct (60.30)

Epoch 1, Iteration 0, loss = 0.9036
Checking accuracy on validation set
Got 612 / 1000 correct (61.20)

Epoch 1, Iteration 100, loss = 1.1446
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should be able to see atleast 55% accuracy

In [ ]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)


Checking accuracy on test set
Got 6059 / 10000 correct (60.59)


60.589999999999996

## Part II Self-Attention

In the next section, you will implement an Attention layer which you will then use within a convnet architecture defined above for cifar 10 classification task.

A self-attention layer is formulated as following:

Input: $X$ of shape $(H\times W, C)$

Query, key, value linear transforms are $W_Q$, $W_K$, $W_V$, of shape $(C, C)$. We implement these linear transforms as 1x1 convolutional layers of the same dimensions.

$XW_Q$, $XW_K$, $XW_V$, represent the output volumes when input X is passed through the transforms.


Self-Attention is given by the formula: $Attention(X) = X + Softmax(\frac{XW_Q(XW_K)^\top}{\sqrt{C}})XW_V$

### Inline Question 1: Self-Attention is equivalent to which of the following: (5 points)
1. K-means clustering <br />
2. Non-local means <br />
3. Residual Block <br />
4. Gaussian Blurring <br />

Your Answer: Non-local means

### Here you implement the Attention module, and run it in the next section (40 points)

In [ ]:
# Initialize the attention module as a nn.Module subclass
class Attention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # TODO: Implement the Key, Query and Value linear transforms as 1x1 convolutional layers
        # Hint: channel size remains constant throughout
        self.conv_query = nn.Conv2d(in_channels, in_channels, 1)
        self.conv_key = nn.Conv2d(in_channels, in_channels, 1)
        self.conv_value = nn.Conv2d(in_channels, in_channels, 1)

    def forward(self, x):
        N, C, H, W = x.shape
        
        # TODO: Pass the input through conv_query, reshape the output volume to (N, C, H*W)
        q = self.conv_query(x).reshape(N, C, H*W)
        # TODO: Pass the input through conv_key, reshape the output volume to (N, C, H*W)
        k = self.conv_key(x).reshape(N, C, H*W)
        # TODO: Pass the input through conv_value, reshape the output volume to (N, C, H*W)
        v = self.conv_value(x).reshape(N, C, H*W)
        # TODO: Implement the above formula for attention using q, k, v, C
        # NOTE: The X in the formula is already added for you in the return line
        attention = torch.matmul(F.softmax(torch.matmul(q,k.transpose(1,2))/np.sqrt(C), dim = 1), v)
        # Reshape the output to (N, C, H, W) before adding to the input volume
        attention = attention.reshape(N, C, H, W)
        return x + attention

## Single Attention Block: Early attention; After the first conv layer. (10 points)

In [ ]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the first Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3196
Checking accuracy on validation set
Got 108 / 1000 correct (10.80)

Epoch 0, Iteration 100, loss = 1.6360
Checking accuracy on validation set
Got 456 / 1000 correct (45.60)

Epoch 0, Iteration 200, loss = 1.8984
Checking accuracy on validation set
Got 488 / 1000 correct (48.80)

Epoch 0, Iteration 300, loss = 1.2470
Checking accuracy on validation set
Got 493 / 1000 correct (49.30)

Epoch 0, Iteration 400, loss = 1.4670
Checking accuracy on validation set
Got 537 / 1000 correct (53.70)

Epoch 0, Iteration 500, loss = 1.3363
Checking accuracy on validation set
Got 562 / 1000 correct (56.20)

Epoch 0, Iteration 600, loss = 1.2304
Checking accuracy on validation set
Got 577 / 1000 correct (57.70)

Epoch 0, Iteration 700, loss = 0.9586
Checking accuracy on validation set
Got 593 / 1000 correct (59.30)

Epoch 1, Iteration 0, loss = 0.9776
Checking accuracy on validation set
Got 601 / 1000 correct (60.10)

Epoch 1, Iteration 100, loss = 1.0467
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should see improvement of about 2-3% over the vanilla convnet model. * Use this part to tune your Attention module and then move on to the next parts. *

In [ ]:
earlyAttention = model
check_accuracy(loader_test, earlyAttention)

Checking accuracy on test set
Got 6073 / 10000 correct (60.73)


60.73

## Single Attention Block: Late attention; After the second conv layer. (10 points)

In [ ]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3014
Checking accuracy on validation set
Got 108 / 1000 correct (10.80)

Epoch 0, Iteration 100, loss = 1.5123
Checking accuracy on validation set
Got 431 / 1000 correct (43.10)

Epoch 0, Iteration 200, loss = 1.6654
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Epoch 0, Iteration 300, loss = 1.0653
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Epoch 0, Iteration 400, loss = 1.3210
Checking accuracy on validation set
Got 571 / 1000 correct (57.10)

Epoch 0, Iteration 500, loss = 1.2393
Checking accuracy on validation set
Got 572 / 1000 correct (57.20)

Epoch 0, Iteration 600, loss = 1.3949
Checking accuracy on validation set
Got 608 / 1000 correct (60.80)

Epoch 0, Iteration 700, loss = 1.0866
Checking accuracy on validation set
Got 609 / 1000 correct (60.90)

Epoch 1, Iteration 0, loss = 0.8586
Checking accuracy on validation set
Got 627 / 1000 correct (62.70)

Epoch 1, Iteration 100, loss = 1.0464
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [ ]:
lateAttention = model
check_accuracy(loader_test, lateAttention)

Checking accuracy on test set
Got 6080 / 10000 correct (60.80)


60.8

### Inline Question 2: Provide one example each of usage of self-attention and attention in computer vision. Explain the difference between the two. (5 points)


Your Answer: Self attention takes all features in a label into account unlike normal attention. Self attention is often used object detection. It has been proven to greatly improve generalization results. Normal attention can take input from multiple layers and is often used for image labeling.

## Double Attention Blocks: After conv layers 1 and 2 (10 points)

In [ ]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3445
Checking accuracy on validation set
Got 128 / 1000 correct (12.80)

Epoch 0, Iteration 100, loss = 1.4827
Checking accuracy on validation set
Got 423 / 1000 correct (42.30)

Epoch 0, Iteration 200, loss = 1.5388
Checking accuracy on validation set
Got 475 / 1000 correct (47.50)

Epoch 0, Iteration 300, loss = 1.1457
Checking accuracy on validation set
Got 514 / 1000 correct (51.40)

Epoch 0, Iteration 400, loss = 1.4062
Checking accuracy on validation set
Got 552 / 1000 correct (55.20)

Epoch 0, Iteration 500, loss = 1.1147
Checking accuracy on validation set
Got 557 / 1000 correct (55.70)

Epoch 0, Iteration 600, loss = 1.1310
Checking accuracy on validation set
Got 565 / 1000 correct (56.50)

Epoch 0, Iteration 700, loss = 1.1542
Checking accuracy on validation set
Got 599 / 1000 correct (59.90)

Epoch 1, Iteration 0, loss = 1.0807
Checking accuracy on validation set
Got 610 / 1000 correct (61.00)

Epoch 1, Iteration 100, loss = 1.0327
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [ ]:
doubleAttention = model
check_accuracy(loader_test, vanillaModel)

Checking accuracy on test set
Got 6059 / 10000 correct (60.59)


60.589999999999996

## Resnet with Attention 

Now we will experiment with applying attention within the Resnet10 architecture that we implemented in Homework 2. Please note that for a deeper model such as Resnet we do not expect significant improvements in performance with Attention

## Vanilla Resnet, No Attention

The architecture for Resnet is given below, please train it and evaluate it on the test set.

In [ ]:
import torch
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, block, layers, img_channels=3, num_classes=100, batchnorm=False, attention = False):
        super(ResNet, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.attention = attention
        self.Att = Attention(128)
        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)
        if self.attention:
            x = self.Att(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x


        

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)
        
class block(nn.Module):

    def __init__(self, in_channels, out_channels, downsampler = None, stride = 1, batchnorm=False):
        
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampler = downsampler
        self.relu = nn.ReLU()
        self.batchnorm = batchnorm

    
    def forward(self, x):

        residual = x
        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        if self.batchnorm:
            x = self.bn2(x)
        x = self.relu(x)
        
        if self.downsampler:
            residual = self.downsampler(residual)

        return self.relu(residual + x)
    


def ResNet10(num_classes = 100, batchnorm= False, attention = False):

    return ResNet(block, [1, 1, 1, 1], num_classes=num_classes, batchnorm=batchnorm, attention = attention)

In [ ]:
model = ResNet10(num_classes = 10, batchnorm = True)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3885
Checking accuracy on validation set
Got 107 / 1000 correct (10.70)

Epoch 0, Iteration 100, loss = 1.3950
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Epoch 0, Iteration 200, loss = 1.6279
Checking accuracy on validation set
Got 464 / 1000 correct (46.40)

Epoch 0, Iteration 300, loss = 1.5916
Checking accuracy on validation set
Got 534 / 1000 correct (53.40)

Epoch 0, Iteration 400, loss = 1.4791
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)

Epoch 0, Iteration 500, loss = 1.1664
Checking accuracy on validation set
Got 557 / 1000 correct (55.70)

Epoch 0, Iteration 600, loss = 1.2149
Checking accuracy on validation set
Got 581 / 1000 correct (58.10)

Epoch 0, Iteration 700, loss = 1.1154
Checking accuracy on validation set
Got 591 / 1000 correct (59.10)

Epoch 1, Iteration 0, loss = 1.0086
Checking accuracy on validation set
Got 588 / 1000 correct (58.80)

Epoch 1, Iteration 100, loss = 0.9786
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [ ]:
vanillaResnet = model
check_accuracy(loader_test, vanillaResnet)

Checking accuracy on test set
Got 7762 / 10000 correct (77.62)


77.62

## Resnet with Attention (5 points)

In [ ]:
## Resnet with Attention

learning_rate = 1e-3

# TODO: Use the above Attention module after the 2nd resnet block i.e. after self.layer2.

model = ResNet10(num_classes = 10, batchnorm = True, attention = True)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3087
Checking accuracy on validation set
Got 151 / 1000 correct (15.10)

Epoch 0, Iteration 100, loss = 1.5221
Checking accuracy on validation set
Got 378 / 1000 correct (37.80)

Epoch 0, Iteration 200, loss = 1.4235
Checking accuracy on validation set
Got 483 / 1000 correct (48.30)

Epoch 0, Iteration 300, loss = 1.3027
Checking accuracy on validation set
Got 511 / 1000 correct (51.10)

Epoch 0, Iteration 400, loss = 1.2844
Checking accuracy on validation set
Got 513 / 1000 correct (51.30)

Epoch 0, Iteration 500, loss = 1.2342
Checking accuracy on validation set
Got 558 / 1000 correct (55.80)

Epoch 0, Iteration 600, loss = 1.1539
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Epoch 0, Iteration 700, loss = 1.0678
Checking accuracy on validation set
Got 585 / 1000 correct (58.50)

Epoch 1, Iteration 0, loss = 0.8666
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Epoch 1, Iteration 100, loss = 1.0750
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [ ]:
AttentionResnet = model
check_accuracy(loader_test, AttentionResnet)

Checking accuracy on test set
Got 8021 / 10000 correct (80.21)


80.21000000000001

## Inline Question 3: Rank the above models based on their performance on test dataset (15 points)
( You are encouraged to run each of the experiments (training) at
least 3 times to get an average estimate )

Report the test accuracies alongside the model names. For example, 1. Vanilla CNN (57.45%, 57.99%).. etc

1. <br />Resnet Attention (80.21%, 78.6%, 79.1%)Avg: 79.3%
2. <br />Resnet Vanilla (78.6%, 79.33%, 77.62%)Avg: 78.52%
3. <br />Double Attention (62.24%, 60.59%, 60.59%)Avg: 61.14%
4. <br />Early Attention (60.14%, 60.3%, 60.73%)Avg: 60.4%
5. <br />Late Attention (59.07%, 61.58%, 60.08%)Avg: 60.1%
6. <br />Vanilla CNN (56.39%, 58.81%, 60.58%) Avg: 58.6%

### Bonus Question (Ungraded): Can you give a possible explanation that supports the rankings?
Your Answer: It is clear that Resnet in general is more accurate for classification but Attention improves all networks. This is clear because double attention is the most accurate for the CNN. Adding attention in Resnet only includes a small increase in accuracy. The attention likely improves object classification because it reweighs certain features to make smaller less obvious features more important when classifiying objects

In [ ]:
!git clone https://github.com/jupyter/nbconvert.git
!cd nbconvert
!pip install -e .
!apt-get install pandoc
!apt-get update
!apt-get install inkscape
!add-apt-repository universe
!add-apt-repository ppa:inkscape.dev/stable
!apt-get update
!apt install inkscape
!apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended

In [3]:
!jupyter nbconvert  '/content/drive/MyDrive/Colab Notebooks/assignment4.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/assignment4.ipynb to html
[NbConvertApp] Writing 482776 bytes to /content/drive/MyDrive/Colab Notebooks/assignment4.html
